# Introduction - Learning how to run inference on a Super Resolution Model

This is testing the process of running a model on the NPU. It was a bit fiddly bit have this working.

In [1]:
import onnxruntime
import math
import numpy as np
import os

def main():
    print(f"[INFO] Available Onnxruntime providers: {onnxruntime.get_available_providers()}")

if __name__ == "__main__":
    main()

# Note: This should list the available providers for the current system specifically looking for QNNExecutionProvider, as this will use the NPU.

[INFO] Available Onnxruntime providers: ['QNNExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']


# Prerequisites

- Install the Qualcomm AI SDK - https://softwarecenter.qualcomm.com/#/catalog/item/qualcomm_ai_engine_direct?type=Tool
- Reinstall the onnxruntime QNN if done before
- Ensure that Visual Studio 2022 has C++ dev setup

If all fails Qualcomm Slack support is excellent.

In [ ]:
execution_provider_option = {
    "backend_path": f"C:\\ai\\qcdll\\QnnHtp.dll",
    "session.enable_htp_fp16_precision": "1",
    "htp_performance_mode": "high_performance",
}

In [ ]:
## Full Code Sample with Super Resolution Model working

In [4]:
import onnx
import onnxruntime as ort
import numpy as np
from PIL import Image
import sys

# Install - pip install onnx onnxruntime onnxruntime-qnn pillow

def preprocess_image(image_path):
    # Load image
    image = Image.open(image_path).convert('RGB')  # Convert to grayscale
    image = image.resize((128, 128))  # Resize to the required input size
    image_data = np.array(image).astype(np.uint8)
    image_data = image_data.transpose(2, 0, 1)  # Change data layout from HWC to CHW
    image_data = np.expand_dims(image_data, axis=0)  # Add batch dimension
    return image_data

def postprocess_output(output_data):
    # Remove batch dimension and change data layout from CHW to HWC
    output_image = output_data.squeeze().transpose(1, 2, 0)
    # Convert to uint8
    output_image = output_image.astype(np.uint8)
    return output_image

def main(image_path):
    # Check available providers
    available_providers = ort.get_available_providers()
    print("Available providers:", available_providers)

    # Load the ONNX model
    model_path = r"C:\\ai\\models\\quicksrnetlarge_quantized.onnx"  # Ensure this file exists in the correct path

    execution_provider_option = {
        "backend_path": f"C:\\ai\\qcdll\\QnnHtp.dll",
        "session.enable_htp_fp16_precision": "1",
        "htp_performance_mode": "high_performance",
    }
    
    # Use QNNExecutionProvider regardless of the available providers
    providers = ['QNNExecutionProvider']
    ort_session = ort.InferenceSession(model_path, providers=providers, provider_options=[execution_provider_option])
    
    # Prepare input data
    input_name = ort_session.get_inputs()[0].name
    input_data = preprocess_image(image_path)

    # Run the model
    result = ort_session.run(None, {input_name: input_data})

    # Post-process the output
    output_image = postprocess_output(result[0])

    # Save the output image
    output_image = Image.fromarray(output_image)
    output_image.save("output_image_notebook.png")
    print("Output image saved as output_image_notebook.png")


if __name__ == "__main__":
    main('example_image_128.jpg')


Available providers: ['QNNExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']
Output image saved as output_image_notebook.png
